In [2]:
import tensorflow as tf
import gym
import numpy as np

env = gym.make('CartPole-v0')

/home/alex/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
[2019-03-14 11:55:00,413] Making new env: CartPole-v0


In [2]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes<10:
    env.render()
    observation,reward,done,_ = env.step(np.random.randint(0,2))
    reward_sum+=reward
    if done:
        random_episodes+=1
        print(reward_sum)
        reward_sum = 0
        env.reset()

13.0
14.0
18.0
78.0
34.0
22.0
15.0
31.0
17.0
14.0


In [3]:
H = 10 #hidden layer
batch_size = 5
learning_rate = 1e-2
D=4
gamma = 0.99

In [4]:

##MLP
observations= tf.placeholder(tf.float32,[None,D],name='input_X')
W1 = tf.get_variable("W1",shape=[D,H],initializer = tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable("W2",shape=[H,1],initializer = tf.contrib.layers.xavier_initializer())
layer2 = tf.nn.relu(tf.matmul(layer1,W2))
score =  tf.matmul(layer1,W2)
probability = tf.nn.sigmoid(score)

In [5]:


#人工设置的虚拟label，二值化
input_y = tf.placeholder(tf.float32,[None,1],name='input_y')
#每个Acation 的潜在价值
advantages = tf.placeholder(tf.float32,name='reward_signal')
loglik = tf.log(input_y*(input_y-probability)+(1-input_y)*(input_y+probability))
#loglik: 当前Action对应概率的对数，与潜在价值相乘，并取负数作为损失
loss = -tf.reduce_mean(loglik*advantages)
#所有可以训练的参数
tvars = tf.trainable_variables()
newGrads = tf.gradients(loss,tvars)

adam = tf.train.AdamOptimizer(learning_rate=learning_rate)
W1Grad = tf.placeholder(tf.float32,name="batch_grad1")
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))



In [6]:

#定义函数discount_rewards
#用来估算Action对应的潜在价值discount_r
#每一个Action的潜在价值，为后一个Action的潜在价值乘以衰减系数gamma再加上其直接获得的reward


#越靠前的潜在价值会越大
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_rewards = 0
    size = r.size
    for t in reversed(range(size)):
        running_rewards = gamma*running_rewards+r[t]
        discounted_r[t] = running_rewards
    return discounted_r


In [7]:
#分别是环境信息observation的列表，定义的label的列表，每一个Action的Reward
xs,ys,drs = [],[],[]
reward_sum = 0
episode_number = 1
running_reward = None
total_episodes = 10000
init = tf.initialize_all_variables()

with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset()
    
    gradBuffer = sess.run(tvars)
    
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad*0
        
    while episode_number <= total_episodes:
        
        #如果batch的平均reward大于1000的时候，调用环境来进行展示
        
        if reward_sum/batch_size > 100 or rendering == True:
            
            env.render()
            rendering = True
        x = np.reshape(observation,[1,D])
        
        tfprob = sess.run(probability,feed_dict={observations:x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        
        xs.append(x)
        y = 1 - action
        ys.append(y)
        
        observation,reward,done,info = env.step(action)
        reward_sum += reward
        
        drs.append(reward)
        
        
        if done:
            episode_number+=1
            #纵向堆叠
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            xs,ys,drs = [],[],[]

            #normalized to a stable distribution
            discounted_epr = discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr) 


            tGrad = sess.run(newGrads,feed_dict={observations:epx,input_y:epy,advantages:epr})

            for ix,grad in enumerate(tGrad):
                gradBuffer[ix]+=grad

                
            if episode_number % batch_size==0:
                sess.run(updateGrads,feed_dict={W1Grad:gradBuffer[0],W2Grad:gradBuffer[1]})
                
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad*0
                
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print ('Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size))
                
                if reward_sum/batch_size>200:
                    print("Task solved in ",episode_number,"episodes!")
                    break
                
                reward_sum = 0
                
            observation = env.reset()
                    
                    

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[2019-03-14 11:55:12,910] From /home/alex/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:118: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Average reward for episode 19.400000.  Total average reward 19.400000.
Average reward for episode 15.400000.  Total average reward 19.360000.
Average reward for episode 15.800000.  Total average reward 19.324400.
Average reward for episode 18.800000.  Total average reward 19.319156.
Average reward for episode 22.400000.  Total average reward 19.349964.
Average reward for episode 15.400000.  Total average reward 19.310465.
Average reward for episode 13.800000.  Total average reward 19.255360.
Average reward for episode 12.400000.  Total average reward 19.186807.
Average reward for episode 21.800000.  Total average reward 19.212938.
Average reward for episode 16.200000.  Total average reward 19.182809.
Average reward for episode 23.200000.  Total average reward 19.222981.
Average reward for episode 11.800000.  Total average reward 19.148751.
Average reward for episode 15.800000.  Total average reward 19.115264.
Average reward for episode 22.600000.  Total average reward 19.150111.
Averag

KeyboardInterrupt: 

In [8]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
        #if reward_sum/batch_size > 100 or rendering == True : 
            #env.render()
            #rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        # 批量更新
        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                #当累积了足够多的梯度的时候才才进行梯度的更新
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print ('Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size))
                
                if reward_sum/batch_size > 200: 
                    print ("Task solved in",episode_number,'episodes!')
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
        


Average reward for episode 15.400000.  Total average reward 15.400000.
Average reward for episode 21.200000.  Total average reward 15.458000.
Average reward for episode 24.000000.  Total average reward 15.543420.
Average reward for episode 29.800000.  Total average reward 15.685986.
Average reward for episode 38.000000.  Total average reward 15.909126.
Average reward for episode 18.400000.  Total average reward 15.934035.
Average reward for episode 27.200000.  Total average reward 16.046694.
Average reward for episode 18.800000.  Total average reward 16.074227.
Average reward for episode 36.000000.  Total average reward 16.273485.
Average reward for episode 24.800000.  Total average reward 16.358750.
Average reward for episode 21.000000.  Total average reward 16.405163.
Average reward for episode 36.800000.  Total average reward 16.609111.
Average reward for episode 27.600000.  Total average reward 16.719020.
Average reward for episode 26.600000.  Total average reward 16.817830.
Averag

ArgumentError: argument 2: <class 'TypeError'>: wrong type

In [9]:
env = gym.make('FrozenLake-v0')

[2019-03-14 13:27:38,354] Making new env: FrozenLake-v0


In [14]:
#下面是一个Ｑ估值网络的ｄｅｍｏ

Q = np.zeros([env.observation_space.n,env.action_space.n])
lr = .85
y = 0.99
num_episodes = 100000
rewardList = []
for i in range(num_episodes):
    s = env.reset()
    reward_sum = 0
    done = False
    j = 0
    while j<99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:]+np.random.randn(1,env.action_space.n)*(1./(i+1)))
        new_status,reward,done,_ = env.step(a)
        
        #update status
        Q[s,a] = (1-lr)*Q[s,a] + lr*(reward+y*np.max(Q[new_status,:]))
        reward_sum+=reward
        s = new_status
        if done==True:
            break
    rewardList.append(reward_sum)
print ("Score over time: " +  str(sum(rewardList)/num_episodes))
print ("Final Q-Table Values")
print (Q)


Score over time: 0.62426
Final Q-Table Values
[[5.57836375e-01 1.13728739e-02 1.01533036e-02 1.12691614e-02]
 [2.61223193e-04 8.35631695e-03 1.80985851e-03 6.02301528e-01]
 [8.14432174e-03 1.63941675e-02 6.06488748e-03 6.47034787e-01]
 [3.64013696e-04 2.36993606e-03 1.36648726e-03 3.72670831e-01]
 [6.63660195e-01 1.71008614e-03 1.88394561e-03 1.40874090e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.18207494e-06 3.66533616e-08 8.10999621e-02 9.21918470e-08]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.34395835e-04 2.06623395e-04 2.24706076e-03 8.11000456e-01]
 [0.00000000e+00 7.95460759e-01 0.00000000e+00 0.00000000e+00]
 [5.80904729e-01 1.86760613e-05 1.38810363e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.13893027e-03 3.49952384e-04 9.21441376e-01 4.86602202e-03]
 [0.00000000e+00 9.93345690e-01 0.00000000e+00 0.00000000e+00]
 [0.00000